In [ ]:
import pandas as pd

# Carregar o dataset
dados = pd.read_csv("Historico_de_materias.csv")

# Exibir as 5 primeiras linhas do dataset
dados.head()


In [ ]:
# Verificação do número de linhas e colunas
dados.shape

In [ ]:
# Removendo todas as linhas onde o assunto não é "esporte"
dados_esporte = dados[~dados['assunto'].isin(['economia', 'política', 'famosos'])]

In [ ]:
# Exibir as 5 primeiras linhas do novo dataframe 'dados_esporte'
dados_esporte.head()

In [ ]:
#Verificação do número de linhas e colunas
dados_esporte.shape


In [ ]:
# Exibe as colunas para entender a estrutura
print(dados_esporte.columns)


In [ ]:
# Exemplo de leitura de uma notícia

# 'conteudo_noticia' é o nome da coluna que armazena o texto
print(dados_esporte['conteudo_noticia'][74])  

In [ ]:
# Carregar o modelo de linguagem do spaCy
import spacy

nlp = spacy.load("pt_core_news_sm")


In [ ]:
# Configurar stopwords
import nltk
from nltk.corpus import stopwords

api_stop_words = set(stopwords.words('portuguese'))
minhas_stop_words = {'a', 'e', 'i', 'o', 'u'}
stop_words = api_stop_words | minhas_stop_words

In [ ]:
# Função para fazer o tratamento de linguagem natural no texto
import re

def tratamento_pln(texto):
    texto = texto.lower()  # Normalizar para minúsculas
    texto = re.sub(r'[^a-zA-Záéíóú\s]', '', texto)  # Remover números e pontuações
    doc = nlp(texto)
    clean_tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]
    clean_text = ' '.join(clean_tokens)
    return clean_text

In [ ]:
# Dividir o texto em chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Certificar-se de que todos os valores em 'conteudo_noticia' são strings, substituindo NaN por strings vazias
dados_esporte['conteudo_noticia'] = dados_esporte['conteudo_noticia'].fillna("").astype(str)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=40, chunk_overlap=10)
chunks = [chunk for noticia in dados_esporte['conteudo_noticia'] for chunk in text_splitter.split_text(noticia)]


In [ ]:
# Carregar o modelo de embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

In [ ]:
# Gerar IDs para cada chunk
uids = [f"noticia_{i}" for i in range(len(chunks))]

In [ ]:
# Conectar ao banco de dados vetorial ChromaDB
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="noticias_vetorial")

In [ ]:
# Adicionar documentos ao banco de dados
collection.add(documents=chunks, embeddings=embeddings, ids=uids)

In [ ]:
# Função para consulta ao banco vetorial
def consulta_vetorial(query_text):
    query_embedding = model.encode([query_text])
    results = collection.query(query_embeddings=query_embedding, n_results=1)
    for i in range(len(results['ids'][0])):
        doc_id = results['ids'][0][i]
        distance = results['distances'][0][i]
        document = results['documents'][0][i]
        print(f"ID: {doc_id}")
        print(f"Distância: {distance}")
        print(f"Documento: {document}")
        print("-" * 40)

In [ ]:
# Exemplo de consulta
consulta_vetorial("termos de busca")